In [ ]:
pip install psycopg2

In [ ]:
pip install geoalchemy2

In [ ]:
!apt install libspatialindex-dev
!pip install osmnx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-c6 amd64 1.9.3-2 [55.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-dev amd64 1.9.3-2 [16.0 kB]
Fetched 319 kB in 1s (503 kB/s)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unp

In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la conexión a la base de datos PostgreSQL
db_name = 'avissadb'
db_user = 'avissa'
db_password = 'Buseta2024*'
db_host = '104.238.249.113'
db_port = '5432'

# Crear la conexión
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Lista de ciudades que componen el área metropolitana de San Salvador
cities = [
    "Apopa, El Salvador",
    "Ayutuxtepeque, El Salvador",
    "Cuscatancingo, El Salvador",
    "Delgado, El Salvador",
    "Ilopango, El Salvador",
    "Mejicanos, El Salvador",
    "Nejapa, El salvador",
    "San Marcos, El Salvador",
    "San Martin, El Salvador",
    "Tonacatepeque, El Salvador",
    "Municipio de Soyapango, El Salvador",
    "San Salvador, El Salvador",
    "Antiguo Cuscatlán, El Salvador",
    "Santa Tecla, El Salvador"
]

# Descargar la red de calles para cada ciudad y combinar
all_nodes = []
all_edges = []

for city in cities:
    G = ox.graph_from_place(city, network_type="drive")
    nodes, edges = ox.graph_to_gdfs(G)
    all_nodes.append(nodes)
    all_edges.append(edges)

# Combinar todos los nodos y bordes en un solo GeoDataFrame
all_nodes_gdf = gpd.GeoDataFrame(pd.concat(all_nodes, ignore_index=True))
all_edges_gdf = gpd.GeoDataFrame(pd.concat(all_edges, ignore_index=True))

# Almacenar los GeoDataFrames en PostgreSQL
all_nodes_gdf.to_postgis('nodes', engine, if_exists='replace', index=False)
all_edges_gdf.to_postgis('edges', engine, if_exists='replace', index=False)

print("Datos de las redes de calles almacenados en la base de datos.")


Datos de las redes de calles almacenados en la base de datos.
